### Sockets

> Существуют клиентские и серверные сокеты. Серверный сокет прослушивает определенный порт, а клиентский подключается к серверу через этот порт.  
> После того, как было установлено соединение начинается обмен данными.

Single connection

In [ ]:
import socket

In [ ]:
# сервер
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as serv:
    serv.bind(('localhost', 5151))
    serv.listen()
    
    print(f'Start server to listen ...')
    conn, addr_client = serv.accept()
    
    with conn:
        print(f'Connected to client [{addr_client}]')
        
        datas = ''
        while True:
            data = conn.recv(1024)

            if not data:
                break
            
            for enc in ('utf-8', 'windows-1251', 'cp1252'):
                try:
                    data = data.decode(encoding=enc)
                    datas += data
                    conn.sendall(f'Server received data'.encode(encoding='utf-8'))
                    break
                except UnicodeDecodeError:
                    continue

    print(f'Disconnect client [{addr_client}]')

In [ ]:
# клиент
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect(('localhost', 5151))
    
    while True:
        msg = input('Enter the command ["exit" to quit]: ')
        if msg.lower() != 'exit':
            s.sendall(f'{msg}'.encode(encoding='utf-8'))
            data = s.recv(1024).decode(encoding='utf-8')
            print(f"Received from server: {data}")
        else:
            s.close()
            break        
    print('Connection closed')

asyncio

In [ ]:
# сервер
import asyncio

async def handle_client(client, addr_client):
    loop = asyncio.get_event_loop()
    request = None
    print(f'Connected to client [{addr_client}]')
    while request != 'exit':
        request = (await loop.sock_recv(client, 1024)).decode('utf-8')
        response = f'Server received data\n'
        await loop.sock_sendall(client, response.encode('utf-8'))
    client.close()


async def run_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 5151))
    server.listen(5)
    server.setblocking(False)
    loop = asyncio.get_event_loop()
    while True:
        client, addr_client = await loop.sock_accept(server)
        loop.create_task(handle_client(client, addr_client))

asyncio.run(run_server())